In [1]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive


In [2]:
%cd '/gdrive/My Drive/VB/NCAA'
%ls

/gdrive/My Drive/VB/NCAA
AllTeamMatchStats.csv  BigTenAllYearBreakdown.csv  MatchStats.ipynb
AllTeams2019Break.csv  BigTenBox.csv               MatchStats.py
AllTeams2019.csv       BigTenBreakdown.csv         ParseNCAA.ipynb
AllTeams2019.ipynb     LMPredictWins.ipynb         ParseNCAA.py
AllTeams2019Sched.csv  MatchStats5Year.csv
AllTeamsConf2019.csv   MatchStats.csv


In [3]:
%pip install skorch

     |████████████████████████████████| 102kB 1.4MB/s 


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [5]:
sched = pd.read_csv('MatchStats5Year.csv')
sched.head()

,Date,Opponent,S,Kills,Errors,Total Attacks,Hit Pct,Assists,Aces,SErr,Digs,RErr,Block Solos,Block Assists,BErr,PTS,BHE,Team,Sets Won,Sets Lost,Match Result,Location
0,08/24/2018,South Carolina,3,27,16,69,0.159,24,4,9,31,6,3,4,0,36.0,1,A&M-Corpus Christi,0,3,L,Neutral
1,08/24/2018,Clemson,3,25,22,93,0.032,25,2,9,29,4,1,4,2,30.0,1,A&M-Corpus Christi,0,3,L,Neutral
2,08/25/2018,ETSU,3,41,16,111,0.225,38,5,6,37,4,0,4,2,48.0,0,A&M-Corpus Christi,0,3,L,Neutral
3,08/28/2018,UTRGV,5,54,29,175,0.143,52,4,8,73,2,0,8,0,62.0,1,A&M-Corpus Christi,2,3,L,Neutral
4,08/31/2018,LSU,3,26,15,103,0.107,24,2,7,28,6,1,2,0,30.0,0,A&M-Corpus Christi,0,3,L,Neutral


In [ ]:
sched = sched.drop(['index', 'MP', 'MS', 'Attend', 'Unnamed: 22'], axis=1)

In [ ]:
temp = sched['Result'].str.split(expand=True)
if len(temp.columns) == 2:
    sched['Match Result'] = temp[0]
    sched[['Sets Won', 'Sets Lost']] = temp[1].str.split('-', expand=True)

elif (len(temp.columns) == 3):
    sched['Sets Won'] = temp[0]
    sched['Sets Lost'] = temp[2]
    sched['Match Result'] = 'W'
    sched[sched['Sets Lost'] > sched['Sets Won']] = 'L'

elif (len(temp.columns) == 4):
    sched['Sets Won'] = temp[1]
    sched['Sets Lost'] = temp[3]
    sched['Match Result'] = temp[0]

sched.drop('Result', axis=1, inplace=True)

# create the home/away columm
sched['Location'] = 'Home'
sched.loc[sched['Opponent'].str.startswith('@'), 'Location'] = 'Away'
sched['Opponent'] = sched['Opponent'].str.lstrip('@ ',)

# handle nuetral locations
neutral_ix = sched[sched['Opponent'].str.contains(' @')].index
if not neutral_ix.empty:
    sched.loc[neutral_ix, 'Opponent'] = sched.loc[neutral_ix, 'Opponent'].str.split('@', expand=True)[0]
    sched.loc[neutral_ix, 'Location'] = 'Neutral'
sched['Opponent'] = sched['Opponent'].str.strip()


In [ ]:
sched.loc[:,'S':'BHE'] = sched.loc[:,'S':'BHE'].apply(pd.to_numeric, errors='coerce')
sched.loc[:,'S':'BHE'] = sched.loc[:,'S':'BHE'].fillna(0)

In [6]:
sched.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51249 entries, 0 to 51248
Data columns (total 22 columns):
Date             51249 non-null object
Opponent         51249 non-null object
S                51249 non-null int64
Kills            51249 non-null int64
Errors           51249 non-null int64
Total Attacks    51249 non-null int64
Hit Pct          51249 non-null float64
Assists          51249 non-null int64
Aces             51249 non-null int64
SErr             51249 non-null int64
Digs             51249 non-null int64
RErr             51249 non-null int64
Block Solos      51249 non-null int64
Block Assists    51249 non-null int64
BErr             51249 non-null int64
PTS              51249 non-null float64
BHE              51249 non-null int64
Team             51249 non-null object
Sets Won         51249 non-null int64
Sets Lost        51249 non-null int64
Match Result     51249 non-null object
Location         51249 non-null object
dtypes: float64(2), int64(15), object(5)
memor

In [7]:
%pip install eli5

     |████████████████████████████████| 102kB 2.0MB/s 


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
import eli5

from skorch import NeuralNetClassifier

In [ ]:
sched = sched.drop(['Assists'], axis=1)

In [22]:
X = sched.loc[:,'S':'BHE'].values
X = preprocessing.scale(X)

Y = (sched['Match Result'] == 'W').astype('int').values

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state=42)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(38436, 14) (12813, 14) (38436,) (12813,)


In [25]:
clf = svm.SVC(kernel='rbf', gamma='scale', random_state = 0)

# fit model to training data
clf.fit(X_train, y_train)

# predict using the test data
y_train_pred = clf.predict(X_train)
y_test_pred = clf.predict(X_test)

print('Train acc = %6.3f' % np.mean(np.equal(y_train_pred, y_train)))
print('Test  acc = %6.3f' % np.mean(np.equal(y_test_pred, y_test)))


Train acc =  0.886
Test  acc =  0.864


In [ ]:
from eli5.sklearn import PermutationImportance
perm = PermutationImportance(clf).fit(X_test, y_test)

In [13]:
eli5.show_weights(perm, feature_names=sched.loc[:,'S':'BHE'].columns.values)

Weight,Feature
0.1209 ± 0.0070,Total Attacks
0.1132 ± 0.0035,Hit Pct
0.0877 ± 0.0051,Digs
0.0322 ± 0.0046,Aces
0.0251 ± 0.0022,RErr
0.0242 ± 0.0031,PTS
0.0160 ± 0.0029,Kills
0.0142 ± 0.0024,S
0.0125 ± 0.0026,Block Assists
0.0023 ± 0.0015,Errors


In [23]:
rfc =  RandomForestClassifier(n_estimators=20)

# fit model to training data
rfc.fit(X_train, y_train)

# predict using the test data
y_train_pred = rfc.predict(X_train)
y_test_pred = rfc.predict(X_test)

print('Train acc = %6.3f' % np.mean(np.equal(y_train_pred, y_train)))
print('Test  acc = %6.3f' % np.mean(np.equal(y_test_pred, y_test)))



Train acc =  0.998
Test  acc =  0.848


In [15]:
perm = PermutationImportance(rfc).fit(X_test, y_test)
eli5.show_weights(perm, feature_names=sched.loc[:,'S':'BHE'].columns.values)

Weight,Feature
0.0816 ± 0.0059,Hit Pct
0.0667 ± 0.0030,PTS
0.0229 ± 0.0051,RErr
0.0215 ± 0.0050,Total Attacks
0.0214 ± 0.0030,Digs
0.0151 ± 0.0037,Aces
0.0096 ± 0.0040,S
0.0082 ± 0.0018,Kills
0.0045 ± 0.0027,Errors
0.0013 ± 0.0034,Block Assists


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
class Linear(nn.Module):

    def __init__(self):
        super(Linear, self).__init__()
        
        self.fc1 = nn.Linear(14, 100)
        self.fc2 = nn.Linear(100, 10)
        self.fc3 = nn.Linear(10, 2)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)

        return x

In [ ]:
torch.manual_seed(0)
device = torch.device("cuda")

y_train_tensor = torch.from_numpy(y_train).to(device)

In [19]:
net = NeuralNetClassifier(
    Linear,
    max_epochs=50,
    lr=0.001,
    optimizer=torch.optim.Adam,
    criterion=torch.nn.CrossEntropyLoss,
    device=device,
)

net.fit(X_train.astype('float32'), y_train_tensor)

  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        0.3489       0.8673        0.2753  0.8786
      2        0.2661       0.8729        0.2641  0.8045
      3        0.2589       0.8732        0.2612  0.8291
      4        0.2560       0.8740        0.2602  0.7963
      5        0.2545       0.8737        0.2599  0.7903
      6        0.2535       0.8743        0.2597  0.7916
      7        0.2528       0.8742        0.2597  0.7869
      8        0.2522       0.8737        0.2597  0.8020
      9        0.2517       0.8737        0.2596  0.8188
     10        0.2513       0.8742        0.2597  0.8038
     11        0.2509       0.8742        0.2596  0.8046
     12        0.2505       0.8746        0.2597  0.8000
     13        0.2502       0.8745        0.2598  0.8263
     14        0.2498       0.8743        0.2599  0.8068
     15        0.2495       0.8740        0.2600  0.8161
     16        0.2492       0.8

<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=Linear(
    (fc1): Linear(in_features=14, out_features=100, bias=True)
    (fc2): Linear(in_features=100, out_features=10, bias=True)
    (fc3): Linear(in_features=10, out_features=2, bias=True)
  ),
)

In [20]:
# predict using the test data
y_train_pred = net.predict(X_train.astype('float32'))
y_test_pred = net.predict(X_test.astype('float32'))

print('Train acc = %6.3f' % np.mean(np.equal(y_train_pred, y_train)))
print('Test  acc = %6.3f' % np.mean(np.equal(y_test_pred, y_test)))

Train acc =  0.882
Test  acc =  0.861


In [21]:
perm = PermutationImportance(net).fit(X_test.astype('float32'), y_test)
eli5.show_weights(perm, feature_names=sched.loc[:,'S':'BHE'].columns.values)

Weight,Feature
0.1513 ± 0.0062,Total Attacks
0.1124 ± 0.0056,Digs
0.0872 ± 0.0038,Hit Pct
0.0521 ± 0.0020,Kills
0.0326 ± 0.0040,Aces
0.0182 ± 0.0021,RErr
0.0145 ± 0.0011,S
0.0145 ± 0.0027,Block Assists
0.0068 ± 0.0022,PTS
0.0044 ± 0.0029,Block Solos
